# Chapter 9: NLP with TensorFlow – Sentiment Analysis

Bab ini mengeksplorasi bagaimana Natural Language Processing (NLP) digunakan untuk mengklasifikasikan teks berdasarkan sentimen (positif/negatif). Fokus utama adalah pada pembersihan data teks dan penggunaan memori jangka panjang melalui LSTM.

---

## 9.1 Preprocessing Teks (Pembersihan Noise)
Data teks mentah sangat kotor dan tidak terstruktur. Langkah pembersihan meliputi:
* **Normalisasi**: Mengecilkan huruf (*lowercasing*) dan menghapus tanda baca unik.
* **Stopwords Removal**: Menghapus kata-kata umum yang tidak informatif (seperti "dan", "yang", "di").
* **Lemmatization/Stemming**: Mengonversi kata ke bentuk dasarnya (misal: "berlari" menjadi "lari").
* **Masalah Class Imbalance**: Seringkali jumlah ulasan positif jauh lebih banyak daripada negatif. Hal ini harus diseimbangkan agar model tidak bias.

---

## 9.2 Splitting & Strategi Dataset
Untuk memastikan evaluasi yang jujur, dataset dibagi secara ketat:
* **Validation & Test Set**: Harus dibuat seimbang (*balanced*) agar metrik akurasi mencerminkan kemampuan model yang sebenarnya.
* **Training Set**: Berisi sisa data untuk proses pembelajaran model.

---

## 9.3 Analisis Vocabulary & Panjang Sekuens
Dua hyperparameter kunci dalam NLP adalah ukuran kosakata dan panjang kalimat:
* **Vocabulary Analysis**: Menggunakan `collections.Counter` untuk menghitung frekuensi kata. Kita perlu membatasi jumlah kata unik (misal: hanya mengambil 10.000 kata terpopuler) untuk efisiensi memori.
* **Sequence Length**: Menggunakan `pd.Series.str.len()` untuk melihat distribusi panjang kalimat. Kalimat yang terlalu panjang akan dipotong, dan yang terlalu pendek akan ditambah nol (*padding*).

---

## 9.4 Tokenisasi: Mengubah Teks ke Angka
Komputer hanya memproses angka, sehingga teks harus diubah menjadi ID numerik.
* **Tokenizer Keras**: Memetakan setiap kata unik menjadi integer.
* **OOD (Out of Vocabulary)**: Menyiapkan token khusus (seperti `<OOV>`) untuk menangani kata-kata yang tidak ada dalam daftar kosakata saat fase testing.

---

## 9.5 NLP Pipeline & Bucketing
Mengolah teks secara efisien memerlukan teknik khusus pada `tf.data`:
* **Bucketing**: Mengelompokkan kalimat dengan panjang yang mirip ke dalam satu *batch*. Ini mencegah pemborosan komputasi akibat terlalu banyak *padding* nol pada kalimat pendek yang dipasangkan dengan kalimat sangat panjang.



---

## 9.6 Arsitektur LSTM (Long Short-Term Memory)
LSTM adalah jenis RNN yang dirancang untuk mengingat konteks jarak jauh dalam sebuah kalimat.
* **Dimensi Input**: Memerlukan format `(Batch, Time Steps, Features)`.
* **Mekanisme**: Menggunakan *state vectors* untuk menyimpan memori dari kata-kata sebelumnya.
* **Layer Stack**: Biasanya disusun dengan urutan: `Embedding` -> `Masking` (mengabaikan padding) -> `LSTM` -> `Dropout` -> `Dense` (dengan aktivasi Sigmoid untuk output biner).



---

## 9.7 Word Embeddings (Injeksi Semantik)
Embedding adalah teknik paling modern untuk mewakili kata.
* **Vektor Padat**: Setiap kata diwakili oleh vektor angka (misal: 128 dimensi).
* **Hubungan Semantik**: Kata dengan makna mirip (seperti "Raja" dan "Pangeran") akan memiliki posisi vektor yang berdekatan dalam ruang matematis.
* **Efisiensi**: Ukuran vektor tetap, sehingga penggunaan memori jauh lebih hemat dibandingkan teknik *One-Hot Encoding* yang sangat boros ruang.



---

## 9.8 Tip Implementasi di Colab
Untuk NLP, penggunaan `TextVectorization` layer sangat disarankan di TensorFlow 2:

```python
from tensorflow.keras.layers import TextVectorization

# Mendefinisikan layer standarisasi teks
vectorize_layer = TextVectorization(
    max_tokens=10000,
    output_mode='int',
    output_sequence_length=250)

# Adaptasi dengan data teks
vectorize_layer.adapt(train_text_data)